<img src="credits3.png" style="width:95%;">

<h1 style="color:#0e535b; text-align: left">Banco Uno</h1>

<p style="text-align:justify">En este notebook se pretrabajarán los datos del caso cuyo propósito final será intentar predecir la probabilidad de cumplimiento de pago de los clientes que tienen un crédito en el Banco Uno.</p>


<h3 style="color:#0e535b; text-align: left">1. Definiendo el Proceso para el diseño del algoritmo de Machine Learning</h3>

<h4 style="color:#0e535b; text-align: left">1.1 Carga de Liberías</h4>

<p style="text-align:justify">Se utilizarán librerías que permiten acceder a los datos (sqlalchemy y pymysql) así como librería de que permiten analizar los datos (pandas) y generar informe EDA automático (pandas profiling).</p>


In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import ydata_profiling
from ydata_profiling import ProfileReport

<h4 style="color:#0e535b; text-align: left">1.2 Conexión a base de datos: Creación de String de Conexión</h4>
<p style="text-align:justify">Los datos se encuentran disponibles en una base de datos virtual en un servidor remoto, es por ello que se debe incluir lineas que permiten configurar el acceso a dicha base de datos.</p>

In [2]:
connection = pymysql.connect(host='data-analytics-2018.cbrosir2cswx.us-east-1.rds.amazonaws.com',
user='deepAnalytics',
password='Sqltask1234!',
database='Credit',
charset='utf8mb4',
cursorclass=pymysql.cursors.DictCursor)

<p style="text-align:justify">Se asigna la base de datos generada a un DataFrame llamado df.</p>

In [3]:
df = pd.read_sql('SELECT * FROM credit', con=connection)

C:\Users\sergi\AppData\Local\Temp\ipykernel_11032\3787531507.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM credit', con=connection)


<h4 style="color:#0e535b; text-align: left">1.3 Información de los datos</h4>

<p style="text-align:justify">A continuación se muestran los encabezados y otra información relevante del DataFrame, esta información es una consulta que permitirá conocer cuál tratamiento necesitan los datos.</p>

In [4]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,female,university,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,default
2,120000,female,university,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
3,90000,female,university,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,not default
4,50000,female,university,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,not default


In [5]:
df.tail()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
3665,220000,female,university,1,32,0,0,0,0,0,...,208355,213015,217475,7200,9000,10000,8000,8010,8500,not default
3666,70000,female,university,2,34,1,2,2,2,0,...,26456,28361,31873,1500,2900,0,2500,4000,0,not default
3667,120000,male,university,2,37,-1,2,0,0,0,...,17901,19608,19143,1000,1600,800,2000,0,1600,default
3668,180000,female,university,2,32,0,0,0,0,0,...,31057,29052,25933,1582,30000,1000,1000,1000,1000,not default
3669,50000,female,high school,1,57,0,0,0,0,0,...,48319,48449,49656,2500,2000,2000,1746,2000,1800,not default


In [6]:
df.shape

(3670, 24)

In [7]:
df.columns.values

array(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20',
       'X21', 'X22', 'X23', 'Y'], dtype=object)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3670 entries, 0 to 3669
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   X1      3670 non-null   object
 1   X2      3670 non-null   object
 2   X3      3670 non-null   object
 3   X4      3670 non-null   object
 4   X5      3670 non-null   object
 5   X6      3670 non-null   object
 6   X7      3670 non-null   object
 7   X8      3670 non-null   object
 8   X9      3670 non-null   object
 9   X10     3670 non-null   object
 10  X11     3670 non-null   object
 11  X12     3670 non-null   object
 12  X13     3670 non-null   object
 13  X14     3670 non-null   object
 14  X15     3670 non-null   object
 15  X16     3670 non-null   object
 16  X17     3670 non-null   object
 17  X18     3670 non-null   object
 18  X19     3670 non-null   object
 19  X20     3670 non-null   object
 20  X21     3670 non-null   object
 21  X22     3670 non-null   object
 22  X23     3670 non-null   

In [9]:
df.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
count,3670,3670,3670,3670,3670,3670,3670,3670,3670,3670,...,3670,3670,3670,3670,3670,3670,3670,3670,3670,3670
unique,63,3,5,5,53,10,11,11,10,9,...,2009,1984,1948,1147,1130,1041,1035,1039,971,3
top,50000,female,university,2,29,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,not default
freq,453,2130,1644,2045,214,1741,1901,1875,1995,1996,...,424,460,532,667,708,798,808,827,949,2873


<h4 style="color:#0e535b; text-align: left">1.4. Exportación de los datos a CSV</h4>

<p style="text-align:justify">Se procede a guardar el DataFrame en un documento dentro de la memoria de la computadora.</p>

In [10]:
df.to_csv ('BancoUno.csv' , header = False , index = False )

<h3 style="color:#0e535b; text-align: left">2. Preparando los datos importados</h3>

<h4 style="color:#0e535b; text-align: left">2.1 Renombrar en DataFrame</h4>

<p style="text-align:justify">Se renombrerá el DataFrame, su nuevo nombre se cambia a "credit".</p>

In [11]:
credit = df

<h3 style="color:#0e535b; text-align: left">3. Análisis Exploratorio de los Datos por medio de Pandas Profiling</h3>

<h4 style="color:#0e535b; text-align: left">3.1 Aplicar Pandas Profiling</h4>

<p style="text-align:justify">En este punto se utilizan lineas de código que permitan correr generar un análisis exploratorio de datos (EDA) simplificado por medio de Pandas Profiling, dicho EDA se genera en un documento separado.</p>

In [12]:
profile = ProfileReport(credit, title="Reporte Preliminar")
profile = df.profile_report(title="Pandas Profiling Report")
profile.to_file("BancoUnoPreliminar.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

<h3 style="color:#0e535b; text-align: left">4. Análisis y Exploración de los datos</h3>

<h4 style="color:#0e535b; text-align: left">4.1 Eliminar filas duplicadas</h4>

<p style="text-align:justify">Con base en la exploración de los datos del punto 1.3 y el EDA, se identificaron lineas duplicadas en el DataFrame. Se eliminarán de forma que no alteren los datos para el posterior análisis.</p>

In [13]:
credit.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)
credit.shape

(2397, 24)

In [14]:
credit.head(5)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,female,university,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,default
2,120000,female,university,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
3,90000,female,university,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,not default
4,50000,female,university,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,not default


<h4 style="color:#0e535b; text-align: left">4.2 Ajuste de Encabezados e índices</h4>
<p style="text-align:justify">El DataFrame credit tiene como índice la fila -1, pero esta, no puede ser utilizada como encabezado porque no describe los datos, es por esto que se corregirán los encabezados e índices en el DataFrame utilizando los que corresponden a la siguiente fila.</p>

In [15]:
credit.columns = credit.iloc[0]
credit = credit.iloc[0:]
credit.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object', name=0)

In [16]:
credit.reset_index(inplace=True, drop=True)
credit = credit.drop(0)
credit.head(5)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,female,university,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,default
2,120000,female,university,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
3,90000,female,university,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,not default
4,50000,female,university,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,not default
5,50000,male,university,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,not default


<p style="text-align:justify">Al igual que en el caso anterior, el DataFrame credit tiene un índice de columnas incorrecto que inicia por 1 en lugar de 0, se procede a reestablecer los índices  de columnas y se elimina el indice de columna anterior.</p>

In [17]:
credit.reset_index(inplace=True, drop=False)
credit.drop(['index'], axis=1,inplace=True)
credit.head(5)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,female,university,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,default
1,120000,female,university,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
2,90000,female,university,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,not default
3,50000,female,university,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,not default
4,50000,male,university,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,not default


<h4 style="color:#0e535b; text-align: left">4.3 Normalización de Datos</h4>
<p style="text-align:justify">En esta tarea, se revisa el DataFrame para validar que los datos estén clasificados por su tipo de variable, numérica o categórica.</p>

In [18]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396 entries, 0 to 2395
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   LIMIT_BAL                   2396 non-null   object
 1   SEX                         2396 non-null   object
 2   EDUCATION                   2396 non-null   object
 3   MARRIAGE                    2396 non-null   object
 4   AGE                         2396 non-null   object
 5   PAY_0                       2396 non-null   object
 6   PAY_2                       2396 non-null   object
 7   PAY_3                       2396 non-null   object
 8   PAY_4                       2396 non-null   object
 9   PAY_5                       2396 non-null   object
 10  PAY_6                       2396 non-null   object
 11  BILL_AMT1                   2396 non-null   object
 12  BILL_AMT2                   2396 non-null   object
 13  BILL_AMT3                   2396 non-null   obje

<p style="text-align:justify">Todas las variables tipo object, que requieran un tratamiento numérico deben ser convertidas a tipos integer o float según corresponda. Por la exploración realizada sobre los datos, todas las variables deben ser convertidas, excepto las siguientes cuatro: "SEX", "MARRIAGE", "EDUCATION" y "default payment next month" (a pesar de tener valores numéricos, estas variables son dicotómicas, nominales, u ordinales.. </p>

In [19]:
columnas_int = ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 
        'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 
        'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 
        'PAY_AMT5', 'PAY_AMT6']
credit[columnas_int] = credit[columnas_int].astype(int)

In [20]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396 entries, 0 to 2395
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   LIMIT_BAL                   2396 non-null   int32 
 1   SEX                         2396 non-null   object
 2   EDUCATION                   2396 non-null   object
 3   MARRIAGE                    2396 non-null   object
 4   AGE                         2396 non-null   int32 
 5   PAY_0                       2396 non-null   int32 
 6   PAY_2                       2396 non-null   int32 
 7   PAY_3                       2396 non-null   int32 
 8   PAY_4                       2396 non-null   int32 
 9   PAY_5                       2396 non-null   int32 
 10  PAY_6                       2396 non-null   int32 
 11  BILL_AMT1                   2396 non-null   int32 
 12  BILL_AMT2                   2396 non-null   int32 
 13  BILL_AMT3                   2396 non-null   int3

<h3 style="color:#0e535b; text-align: left">5. Exportar set de datos a formato xls</h3>

In [21]:
credit.to_excel("credit.xlsx")